In [ ]:
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import pytz
import zipline
from matplotlib import style
from six import viewkeys
from trading_calendars import get_calendar
from zipline.api import (
    attach_pipeline,
    date_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
)
from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import RSI

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
try:
    zipline.data.bundles.unregister('hose')
except Exception as e:
    print(e)
    print("# Register the HOSE bundle")
    start_session = pd.Timestamp('2016-01-04', tz='utc')
    end_session = pd.Timestamp('2019-04-22', tz='utc')

    register(
        'hose',
        csvdir_equities(
            ['daily'],
            '/home/doidoi/Documents/cryptocean/csvdir',
        ),
        calendar_name='HOSE',  # HOSE Vietnam
        start_session=start_session,
        end_session=end_session
    )

In [ ]:
# !zipline bundles

In [ ]:
# zipline.data.bundles.bundles

In [ ]:
!zipline clean --bundle hose --before 2099-01-01
!zipline ingest -b hose

In [ ]:
def make_pipeline():
    rsi = RSI()
    return Pipeline(
        columns={
            'longs': rsi.top(3),
            'shorts': rsi.bottom(3),
        },
    )


def rebalance(context, data):
    # Pipeline data will be a dataframe with boolean columns named 'longs' and
    # 'shorts'.
    pipeline_data = context.pipeline_data
    all_assets = pipeline_data.index

    longs = all_assets[pipeline_data.longs]
    shorts = all_assets[pipeline_data.shorts]
    record(universe_size=len(all_assets))
    
    print(context.blotter.current_dt)
    print("longs: " + str(longs))
    print("shorts: " + str(shorts))
    print("portfolio value: " + str(context.portfolio.portfolio_value))
    
    # Build a equal-weight, long-short portfolio.
    one_fourth = 1.0 / 4.0
    for asset in longs:
        order_target_percent(asset, one_fourth)

    for asset in shorts:
        order_target_percent(asset, -one_fourth)

    # Remove any assets that should no longer be in our portfolio.
    portfolio_assets = longs | shorts

    positions = context.portfolio.positions
    for asset in viewkeys(positions) - set(portfolio_assets):
        # This will fail if the asset was removed from our portfolio because it
        # was delisted.
        if data.can_trade(asset):
            order_target_percent(asset, 0)


def initialize(context):
    attach_pipeline(make_pipeline(), name='my_pipeline')
   
    # Rebalance each day.  In daily mode, this is equivalent to putting
    # `rebalance` in our handle_data, but in minute mode, it's equivalent to
    # running at the start of the day each day.
    schedule_function(rebalance, date_rules.every_day())

def handle_data(context, data):
    context.pipeline_data = pipeline_output('my_pipeline')

In [ ]:
perf = zipline.run_algorithm(start=datetime(2016, 4, 1, 0, 0, 0, 0, pytz.timezone('Asia/Ho_Chi_Minh')),
                                 end=datetime(2019, 4, 22, 0, 0, 0, 0, pytz.timezone('Asia/Ho_Chi_Minh')),
                                 initialize=initialize,
                                 handle_data=handle_data,
                                 capital_base=1e9,
                                 trading_calendar=get_calendar('HOSE'),
                                 data_frequency='daily',
                                 default_extension=True,
                                 bundle='hose')

In [ ]:
style.use('seaborn-dark')

dpi = 300

plt.figure(figsize=(1920/dpi, 1080/dpi), dpi=dpi)
ax = perf.portfolio_value.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='portfolio-pecentage-change', color='green')
vals = ax.get_yticks()
ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])

plt.legend(loc=0)
plt.savefig('zipline_vanilla_algo.png', bbox_inches='tight')
plt.show()